# DEPRECATED: Trying to replicate the construction of price datasets from Tennet 
See `2GenerateHistoricalDatasets.ipynb`

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_imbalance = pd.read_csv('data/tennet/settlementprices.csv')
df_mid = pd.read_csv('data/tennet/balancedeltaIGCC.csv')
# https://transparency.entsoe.eu/
df_da = pd.read_csv('./data/entsoe/Day-ahead Prices_201901010000-202001010000.csv')

print("Columns imbalance: ", df_imbalance.columns)
print("Columns mid: ", df_mid.columns)
print("Columns day-ahead: ", df_da.columns)

Columns imbalance:  Index(['Date', 'PTE', 'period_from', 'period_until', 'upward_incident_reserve',
       'downward_incident_reserve', 'To regulate up', 'To regulate down',
       'Incentive component', 'Consume', 'Feed', 'Regulation state'],
      dtype='object')
Columns mid:  Index(['Date', 'Sequence_number', 'Time', 'IGCCContribution_up',
       'IGCCContribution_down', 'To regulate up', 'To regulate down',
       'To regulate up_reserve', 'To regulate down_reserve',
       'Emergency capacity', 'Highest_price_upward', 'Mid_price_upward',
       'Lowest_price_downward'],
      dtype='object')
Columns day-ahead:  Index(['MTU (CET/CEST)', 'Day-ahead Price [EUR/MWh]', 'Currency', 'BZN|NL'], dtype='object')


In [3]:
# Day ahead columns
df_da.describe()
# Skipped

,Day-ahead Price [EUR/MWh],BZN|NL
count,8760.000000,0.0
mean,41.195801,NaN
std,11.278806,NaN
min,-9.020000,NaN
25%,34.100000,NaN
50%,39.700000,NaN
75%,47.392500,NaN
max,121.460000,NaN


In [4]:
# Imbalnace columns
df_imbalance['date'] = df_imbalance.apply(lambda row: pd.to_datetime(f'{row["Date"]} {row["period_until"]}',
                                                                     format='%m/%d/%Y %H:%M'), axis=1)
df_imbalance.rename(columns={'Consume': 'price_imbalance', 'Regulation state': 'regulation_state'}, inplace=True)
print("Process Imbalance")
display(df_imbalance.describe())

Process Imbalance


,PTE,To regulate up,To regulate down,Incentive component,price_imbalance,Feed,regulation_state
count,35040.000000,19971.000000,16910.000000,35040.0,35040.000000,35040.000000,35040.000000
mean,48.500457,63.592258,18.612762,0.0,42.554082,40.391060,0.223459
std,27.712496,62.519171,27.792309,0.0,52.003945,51.227613,1.022846
min,1.000000,0.000000,-487.650000,0.0,-487.650000,-487.650000,-1.000000
25%,24.750000,32.600000,18.000000,0.0,25.530000,24.470000,-1.000000
50%,48.500000,40.630000,24.500000,0.0,32.160000,31.110000,1.000000
75%,72.250000,57.105000,30.200000,0.0,42.540000,40.900000,1.000000
max,100.000000,936.120000,179.010000,0.0,936.120000,936.120000,2.000000


In [5]:
# Mid columns
df_mid['date'] = df_mid.apply(lambda row: pd.to_datetime(f'{row["Date"]} {row["Time"]}',
                                                          format='%m/%d/%Y %H:%M'), axis=1)
df_mid['date']=df_mid['date'].dt.round('15min')
df_mid.rename(columns={'Mid_price_upward': 'mid_price'}, inplace=True)
df_mid = df_mid.drop_duplicates(subset="date")
print("Process mid")
display(df_mid.describe())


Process mid


,Sequence_number,IGCCContribution_up,IGCCContribution_down,To regulate up,To regulate down,To regulate up_reserve,To regulate down_reserve,Emergency capacity,Highest_price_upward,mid_price,Lowest_price_downward
count,35038.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35038.000000,15042.000000,35036.000000,12000.000000
mean,721.527370,35.396935,38.903043,39.567845,23.787733,0.155954,0.002569,0.002454,55.355570,30.478970,22.961194
std,415.693744,75.558363,85.568180,75.163300,55.291123,2.354327,0.256207,0.049483,54.065099,8.065918,21.774535
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.310000,-429.950000
25%,369.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.290000,25.780000,21.057500
50%,729.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,38.080000,29.760000,26.080000
75%,1085.250000,31.000000,32.000000,45.000000,14.000000,0.000000,0.000000,0.000000,50.190000,35.190000,31.780000
max,1494.000000,555.000000,770.000000,559.000000,541.000000,75.000000,35.000000,1.000000,936.120000,107.400000,179.010000


In [6]:
# Joining
df_price = pd.merge(df_imbalance, df_mid, on='date', how='left')
df_price['price_imbalance'] = np.where(df_price['regulation_state']==2, df_price['mid_price'], df_price['price_imbalance'])
df_price["hour"] = df_price["date"].dt.hour
df_price["day_year"] = df_price["date"].dt.dayofyear

df_price = df_price[["date", "hour", "day_year", "price_imbalance"]]

In [7]:
start_date = "01-07-2018"

In [8]:
df_price = df_price.set_index('date')
df_price = df_price[~df_price.index.duplicated(keep='first')]
df_price['price_imbalance'] = df_price['price_imbalance'].fillna(df_price['price_imbalance'].mean())
df_price.describe()

,hour,day_year,price_imbalance
count,35036.000000,35036.000000,35036.000000
mean,11.501056,183.010618,41.146546
std,6.921973,105.368861,50.529675
min,0.000000,1.000000,-487.650000
25%,6.000000,92.000000,25.070000
50%,12.000000,183.000000,31.550000
75%,18.000000,274.000000,41.200000
max,23.000000,365.000000,936.120000


In [9]:
24*365*6

52560

## Final result

In [10]:
df_test = pd.read_csv("./data/2019_test_data_price.csv")
df_test.describe()

,price_da,price_imbalance,Hour
count,8760.000000,8760.000000,8760.000000
mean,0.041197,0.041140,11.500000
std,0.011278,0.034948,6.922582
min,-0.009020,-0.148163,0.000000
25%,0.034100,0.025489,5.750000
50%,0.039700,0.032848,11.500000
75%,0.047393,0.045947,17.250000
max,0.121460,0.460695,23.000000


In [11]:
24*365

8760